# PixiJS Vector Visualization Demo

This notebook demonstrates how to create interactive vector field visualizations using PixiJS.

In [ ]:
# Import necessary libraries
import tempfile
import os
from IPython.display import IFrame, display

## Vector Field Visualization

The visualization below shows a vector field that can be used to represent ocean currents.

In [ ]:
def create_vector_field_template(width=800, height=600, bg_color="0x1099bb"):
    """Create a vector field visualization template"""
    template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Vector Field Demo</title>
        <style>
            body {{ margin: 0; padding: 0; overflow: hidden; }}
        </style>
        <!-- Load PixiJS from CDN -->
        <script src="https://cdn.jsdelivr.net/npm/pixi.js@6.5.2/dist/browser/pixi.min.js"></script>
    </head>
    <body>
        <div id="pixi-container" style="width:{width}px; height:{height}px;"></div>
        
        <div style="position:absolute; top:10px; right:10px; background:rgba(255,255,255,0.8); padding:10px; border-radius:5px;">
            <div>
                <label for="vector-density">Grid Density: </label>
                <input type="range" id="vector-density" min="5" max="40" step="5" value="20">
                <span id="vector-density-value">20</span>
            </div>
            <div>
                <label for="vector-scale">Arrow Size: </label>
                <input type="range" id="vector-scale" min="0.5" max="5" step="0.5" value="2">
                <span id="vector-scale-value">2</span>
            </div>
            <div>
                <label for="field-type">Current Pattern: </label>
                <select id="field-type">
                    <option value="circular">Circular Flow</option>
                    <option value="sink">Convergent</option>
                    <option value="source">Divergent</option>
                    <option value="wave">Wave Pattern</option>
                </select>
            </div>
            <div>
                <label for="color-mode">Coloring: </label>
                <select id="color-mode">
                    <option value="magnitude">By Strength</option>
                    <option value="direction">By Direction</option>
                    <option value="fixed">Single Color</option>
                </select>
            </div>
        </div>
        
        <script>
            // Initialize PixiJS Application
            const app = new PIXI.Application({{
                width: {width},
                height: {height},
                backgroundColor: {bg_color},
                resolution: window.devicePixelRatio || 1,
                antialias: true
            }});
            
            // Add the canvas to the HTML document
            document.getElementById('pixi-container').appendChild(app.view);
            
            // Container for vectors
            const vectorContainer = new PIXI.Container();
            app.stage.addChild(vectorContainer);
            
            // Vector field settings
            let vectorDensity = 20;
            let vectorScale = 2;
            let fieldType = 'circular';
            let colorMode = 'magnitude';
            let time = 0;
            
            // Calculate vector field
            function calculateVector(x, y, type, t=0) {{
                // Normalize coordinates to [-1, 1]
                const nx = (x / app.screen.width) * 2 - 1;
                const ny = (y / app.screen.height) * 2 - 1;
                
                let vx = 0;
                let vy = 0;
                
                switch(type) {{
                    case 'circular':
                        // Circular field
                        vx = -ny;
                        vy = nx;
                        break;
                    case 'sink':
                        // Sink field
                        vx = -nx;
                        vy = -ny;
                        break;
                    case 'source':
                        // Source field
                        vx = nx;
                        vy = ny;
                        break;
                    case 'wave':
                        // Wave field
                        vx = Math.cos(nx * 5 + t);
                        vy = Math.sin(ny * 5 + t);
                        break;
                }}
                
                // Normalize vector
                const mag = Math.sqrt(vx*vx + vy*vy) || 1;
                return {{
                    x: vx / mag,
                    y: vy / mag,
                    magnitude: mag
                }};
            }}
            
            // Color functions
            function getColorByMagnitude(magnitude) {{
                // Color from blue (low) to red (high)
                const r = Math.min(255, Math.floor(magnitude * 255));
                const g = 0;
                const b = Math.min(255, Math.floor(255 - magnitude * 255));
                return (r << 16) | (g << 8) | b;
            }}
            
            function getColorByDirection(vx, vy) {{
                // Color by direction (hue based on angle)
                const angle = Math.atan2(vy, vx);
                const hue = ((angle + Math.PI) / (2 * Math.PI)) * 360;
                
                // Convert HSV to RGB
                const h = hue / 60;
                const s = 1;
                const v = 1;
                
                const i = Math.floor(h);
                const f = h - i;
                const p = v * (1 - s);
                const q = v * (1 - s * f);
                const t = v * (1 - s * (1 - f));
                
                let r, g, b;
                switch (i % 6) {{
                    case 0: r = v; g = t; b = p; break;
                    case 1: r = q; g = v; b = p; break;
                    case 2: r = p; g = v; b = t; break;
                    case 3: r = p; g = q; b = v; break;
                    case 4: r = t; g = p; b = v; break;
                    case 5: r = v; g = p; b = q; break;
                }}
                
                return (Math.floor(r * 255) << 16) | 
                       (Math.floor(g * 255) << 8) | 
                       Math.floor(b * 255);
            }}
            
            // Update vector field
            function updateVectorField() {{
                // Clear the container
                vectorContainer.removeChildren();
                
                // Calculate grid spacing
                const spacing = Math.min(app.screen.width, app.screen.height) / vectorDensity;
                
                // Create vectors
                for (let x = spacing/2; x < app.screen.width; x += spacing) {{
                    for (let y = spacing/2; y < app.screen.height; y += spacing) {{
                        // Calculate vector at this position
                        const vector = calculateVector(x, y, fieldType, time);
                        
                        // Create arrow
                        const arrow = new PIXI.Graphics();
                        
                        // Determine color based on mode
                        let color;
                        if (colorMode === 'magnitude') {{
                            color = getColorByMagnitude(vector.magnitude);
                        }} else if (colorMode === 'direction') {{
                            color = getColorByDirection(vector.x, vector.y);
                        }} else {{
                            color = 0xFFFFFF;  // Fixed white color
                        }}
                        
                        // Draw arrow
                        const length = spacing * 0.8 * vectorScale;
                        const headLength = length * 0.3;
                        const headAngle = Math.PI / 6;
                        
                        // Calculate endpoint
                        const endX = vector.x * length;
                        const endY = vector.y * length;
                        
                        // Draw line
                        arrow.lineStyle(2, color);
                        arrow.moveTo(0, 0);
                        arrow.lineTo(endX, endY);
                        
                        // Calculate angle for arrowhead
                        const angle = Math.atan2(endY, endX);
                        
                        // Draw arrowhead
                        arrow.lineTo(
                            endX - headLength * Math.cos(angle - headAngle),
                            endY - headLength * Math.sin(angle - headAngle)
                        );
                        arrow.moveTo(endX, endY);
                        arrow.lineTo(
                            endX - headLength * Math.cos(angle + headAngle),
                            endY - headLength * Math.sin(angle + headAngle)
                        );
                        
                        // Position arrow
                        arrow.x = x;
                        arrow.y = y;
                        
                        vectorContainer.addChild(arrow);
                    }}
                }}
            }}
            
            // Animation ticker
            app.ticker.add(() => {{
                time += 0.01;
                updateVectorField();
            }});
            
            // Initial update
            updateVectorField();
            
            // Setup control event listeners
            document.getElementById('vector-density').addEventListener('input', function(e) {{
                vectorDensity = parseInt(e.target.value);
                document.getElementById('vector-density-value').textContent = vectorDensity;
            }});
            
            document.getElementById('vector-scale').addEventListener('input', function(e) {{
                vectorScale = parseFloat(e.target.value);
                document.getElementById('vector-scale-value').textContent = vectorScale;
            }});
            
            document.getElementById('field-type').addEventListener('change', function(e) {{
                fieldType = e.target.value;
            }});
            
            document.getElementById('color-mode').addEventListener('change', function(e) {{
                colorMode = e.target.value;
            }});
        </script>
    </body>
    </html>
    """
    
    return template

def display_vector_field(width=800, height=600, bg_color="0x333333"):
    """Display vector field visualization"""
    html = create_vector_field_template(width, height, bg_color)
    
    # Save to temporary file
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "pixi_vector_field.html")
    
    with open(temp_path, "w") as f:
        f.write(html)
    
    return IFrame(src=temp_path, width=width, height=height)

In [ ]:
# Display the vector field visualization
display(display_vector_field(width=900, height=700, bg_color="0x003366"))

## Adding a Map Background

For a more realistic visualization, you can add a map background to the vector field. This would be ideal for displaying ocean currents in a specific geographic region.

In [ ]:
def create_map_vector_template(width=800, height=600, center_lat=-12.46, center_lon=130.84, zoom=8):
    """Create a vector field with map background template"""
    template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Vector Field with Map</title>
        <style>
            body {{ margin: 0; padding: 0; overflow: hidden; }}
            #map-container {{ 
                width: {width}px; 
                height: {height}px; 
                position: relative; 
            }}
            .controls {{
                position: absolute;
                top: 10px;
                right: 10px;
                background: rgba(255,255,255,0.8);
                padding: 10px;
                border-radius: 5px;
                z-index: 1000;
            }}
        </style>
        <!-- Load PixiJS and Leaflet -->
        <script src="https://cdn.jsdelivr.net/npm/pixi.js@6.5.2/dist/browser/pixi.min.js"></script>
        <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
        <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    </head>
    <body>
        <div id="map-container">
            <div class="controls">
                <div>
                    <label for="vector-density">Grid Density: </label>
                    <input type="range" id="vector-density" min="5" max="40" step="5" value="20">
                    <span id="vector-density-value">20</span>
                </div>
                <div>
                    <label for="vector-scale">Arrow Size: </label>
                    <input type="range" id="vector-scale" min="0.5" max="5" step="0.5" value="2">
                    <span id="vector-scale-value">2</span>
                </div>
                <div>
                    <label for="field-type">Current Pattern: </label>
                    <select id="field-type">
                        <option value="circular">Circular Flow</option>
                        <option value="sink">Convergent</option>
                        <option value="source">Divergent</option>
                        <option value="wave">Wave Pattern</option>
                    </select>
                </div>
                <div>
                    <label for="color-mode">Coloring: </label>
                    <select id="color-mode">
                        <option value="magnitude">By Strength</option>
                        <option value="direction">By Direction</option>
                        <option value="fixed">Single Color</option>
                    </select>
                </div>
            </div>
        </div>
        
        <script>
            // Initialize Leaflet map
            const map = L.map('map-container').setView([{center_lat}, {center_lon}], {zoom});
            
            // Add tile layer
            L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
                attribution: '&copy; OpenStreetMap contributors'
            }}).addTo(map);
            
            // Create PixiJS overlay
            const pixiContainer = document.createElement('div');
            pixiContainer.style.position = 'absolute';
            pixiContainer.style.top = '0';
            pixiContainer.style.left = '0';
            pixiContainer.style.pointerEvents = 'none';
            map._container.appendChild(pixiContainer);
            
            // Initialize PixiJS
            const app = new PIXI.Application({{
                width: {width},
                height: {height},
                transparent: true,
                antialias: true
            }});
            pixiContainer.appendChild(app.view);
            
            // Container for vectors
            const vectorContainer = new PIXI.Container();
            app.stage.addChild(vectorContainer);
            
            // Vector field settings
            let vectorDensity = 20;
            let vectorScale = 2;
            let fieldType = 'circular';
            let colorMode = 'magnitude';
            let time = 0;
            
            // Calculate vector field
            function calculateVector(x, y, type, t=0) {{
                // Normalize coordinates to [-1, 1]
                const nx = (x / app.screen.width) * 2 - 1;
                const ny = (y / app.screen.height) * 2 - 1;
                
                let vx = 0;
                let vy = 0;
                
                switch(type) {{
                    case 'circular':
                        // Circular field
                        vx = -ny;
                        vy = nx;
                        break;
                    case 'sink':
                        // Sink field
                        vx = -nx;
                        vy = -ny;
                        break;
                    case 'source':
                        // Source field
                        vx = nx;
                        vy = ny;
                        break;
                    case 'wave':
                        // Wave field
                        vx = Math.cos(nx * 5 + t);
                        vy = Math.sin(ny * 5 + t);
                        break;
                }}
                
                // Normalize vector
                const mag = Math.sqrt(vx*vx + vy*vy) || 1;
                return {{
                    x: vx / mag,
                    y: vy / mag,
                    magnitude: mag
                }};
            }}
            
            // Color functions
            function getColorByMagnitude(magnitude) {{
                // Color from blue (low) to red (high)
                const r = Math.min(255, Math.floor(magnitude * 255));
                const g = 0;
                const b = Math.min(255, Math.floor(255 - magnitude * 255));
                return (r << 16) | (g << 8) | b;
            }}
            
            function getColorByDirection(vx, vy) {{
                // Color by direction (hue based on angle)
                const angle = Math.atan2(vy, vx);
                const hue = ((angle + Math.PI) / (2 * Math.PI)) * 360;
                
                // Convert HSV to RGB
                const h = hue / 60;
                const s = 1;
                const v = 1;
                
                const i = Math.floor(h);
                const f = h - i;
                const p = v * (1 - s);
                const q = v * (1 - s * f);
                const t = v * (1 - s * (1 - f));
                
                let r, g, b;
                switch (i % 6) {{
                    case 0: r = v; g = t; b = p; break;
                    case 1: r = q; g = v; b = p; break;
                    case 2: r = p; g = v; b = t; break;
                    case 3: r = p; g = q; b = v; break;
                    case 4: r = t; g = p; b = v; break;
                    case 5: r = v; g = p; b = q; break;
                }}
                
                return (Math.floor(r * 255) << 16) | 
                       (Math.floor(g * 255) << 8) | 
                       Math.floor(b * 255);
            }}
            
            // Update vector field
            function updateVectorField() {{
                // Clear the container
                vectorContainer.removeChildren();
                
                // Calculate grid spacing based on current view
                const spacing = Math.min(app.screen.width, app.screen.height) / vectorDensity;
                
                // Create grid of vectors
                for (let x = spacing/2; x < app.screen.width; x += spacing) {{
                    for (let y = spacing/2; y < app.screen.height; y += spacing) {{
                        // Calculate vector at this position
                        const vector = calculateVector(x, y, fieldType, time);
                        
                        // Create arrow
                        const arrow = new PIXI.Graphics();
                        
                        // Determine color based on mode
                        let color;
                        if (colorMode === 'magnitude') {{
                            color = getColorByMagnitude(vector.magnitude);
                        }} else if (colorMode === 'direction') {{
                            color = getColorByDirection(vector.x, vector.y);
                        }} else {{
                            color = 0xFFFFFF;  // Fixed white color
                        }}
                        
                        // Draw arrow
                        const length = spacing * 0.8 * vectorScale;
                        const headLength = length * 0.3;
                        const headAngle = Math.PI / 6;
                        
                        // Calculate endpoint
                        const endX = vector.x * length;
                        const endY = vector.y * length;
                        
                        // Draw line
                        arrow.lineStyle(2, color);
                        arrow.moveTo(0, 0);
                        arrow.lineTo(endX, endY);
                        
                        // Calculate angle for arrowhead
                        const angle = Math.atan2(endY, endX);
                        
                        // Draw arrowhead
                        arrow.lineTo(
                            endX - headLength * Math.cos(angle - headAngle),
                            endY - headLength * Math.sin(angle - headAngle)
                        );
                        arrow.moveTo(endX, endY);
                        arrow.lineTo(
                            endX - headLength * Math.cos(angle + headAngle),
                            endY - headLength * Math.sin(angle + headAngle)
                        );
                        
                        // Position arrow
                        arrow.x = x;
                        arrow.y = y;
                        
                        vectorContainer.addChild(arrow);
                    }}
                }}
            }}
            
            // Animation ticker
            app.ticker.add(() => {{
                time += 0.01;
                updateVectorField();
            }});
            
            // Update when map moves
            map.on('move', function() {{
                app.renderer.resize(map._container.clientWidth, map._container.clientHeight);
                updateVectorField();
            }});
            
            // Initial update
            updateVectorField();
            
            // Setup control event listeners
            document.getElementById('vector-density').addEventListener('input', function(e) {{
                vectorDensity = parseInt(e.target.value);
                document.getElementById('vector-density-value').textContent = vectorDensity;
            }});
            
            document.getElementById('vector-scale').addEventListener('input', function(e) {{
                vectorScale = parseFloat(e.target.value);
                document.getElementById('vector-scale-value').textContent = vectorScale;
            }});
            
            document.getElementById('field-type').addEventListener('change', function(e) {{
                fieldType = e.target.value;
            }});
            
            document.getElementById('color-mode').addEventListener('change', function(e) {{
                colorMode = e.target.value;
            }});
        </script>
    </body>
    </html>
    """
    
    return template

def display_map_vectors(width=800, height=600, center_lat=-12.46, center_lon=130.84, zoom=8):
    """Display vector field on a map"""
    html = create_map_vector_template(width, height, center_lat, center_lon, zoom)
    
    # Save to temporary file
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "pixi_map_vectors.html")
    
    with open(temp_path, "w") as f:
        f.write(html)
    
    return IFrame(src=temp_path, width=width, height=height)

In [ ]:
# Display the vector field on a map centered on Darwin, Australia
display(display_map_vectors(width=900, height=700, center_lat=-12.46, center_lon=130.84, zoom=8))

## Applications for USV Mission Planning

These interactive visualizations can be adapted to support several aspects of USV mission planning:

1. **Current Visualization**: Display Copernicus current data as vector fields overlaid on maps
2. **Route Planning**: Show optimal paths between survey sites with current-aware routing
3. **Energy Estimation**: Color routes by estimated energy consumption or travel time
4. **Station-keeping Analysis**: Visualize the energy required to maintain position at each site

The map-based visualization is particularly useful as it allows interactive exploration of the Darwin/Beagle Gulf region while displaying the current data directly on the geographic context.